# 1 Understand the Data Structure

In [3]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('../data/raw')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


### 1.1 Check how many unique stores, countries, and product exist

In [4]:
# Stores
print(
    f"Train: \n{train.store.value_counts()}", 
    f"\n====\nTest:\n {test.store.value_counts()}"
    )

Train: 
store
Discount Stickers       76710
Stickers for Less       76710
Premium Sticker Mart    76710
Name: count, dtype: int64 
====
Test:
 store
Discount Stickers       32850
Stickers for Less       32850
Premium Sticker Mart    32850
Name: count, dtype: int64


In [5]:
# Countries
print(
    f"Train: \n{train.country.value_counts()}", 
    f"\n====\nTest:\n {test.country.value_counts()}"
    )

Train: 
country
Canada       38355
Finland      38355
Italy        38355
Kenya        38355
Norway       38355
Singapore    38355
Name: count, dtype: int64 
====
Test:
 country
Canada       16425
Finland      16425
Italy        16425
Kenya        16425
Norway       16425
Singapore    16425
Name: count, dtype: int64


In [6]:
# Products 
print(
    f"Train: \n{train['product'].value_counts()}", 
    f"\n====\nTest:\n {test['product'].value_counts()}"
    )

Train: 
product
Holographic Goose     46026
Kaggle                46026
Kaggle Tiers          46026
Kerneler              46026
Kerneler Dark Mode    46026
Name: count, dtype: int64 
====
Test:
 product
Holographic Goose     19710
Kaggle                19710
Kaggle Tiers          19710
Kerneler              19710
Kerneler Dark Mode    19710
Name: count, dtype: int64


<div style="padding: 20px 12px; background-color: lightcyan; color: darkcyan; font-weight: bold"> The stores, countries, and products are divided equally... </div>

### 1.2 Identify the time range of training vs test data

In [7]:
# Dates
print(
    f"Train: \n{train['date'].agg(['min', 'max'])}", 
    f"\n====\nTest:\n {test['date'].agg(['min', 'max'])}"
    )

Train: 
min    2010-01-01
max    2016-12-31
Name: date, dtype: object 
====
Test:
 min    2017-01-01
max    2019-12-31
Name: date, dtype: object


<div style="padding: 20px 12px; background-color: lightcyan; color: darkcyan; font-weight: bold">
    The time range for training set is from the beginning of 2010 to the end of 2016 (7 years) while for the test set, it is from 2017 to the end of 2019 (2 years).
</div>

### 1.3 Verify date frequency

In [8]:
train['date'].value_counts()

date
2016-12-31    90
2010-01-01    90
2010-01-02    90
2010-01-03    90
2010-01-04    90
              ..
2010-01-18    90
2010-01-17    90
2010-01-16    90
2010-01-15    90
2010-01-14    90
Name: count, Length: 2557, dtype: int64

In [9]:
test['date'].value_counts()

date
2019-12-31    90
2017-01-01    90
2017-01-02    90
2017-01-03    90
2017-01-04    90
              ..
2017-01-12    90
2017-01-11    90
2017-01-10    90
2017-01-09    90
2017-01-08    90
Name: count, Length: 1095, dtype: int64

<div style="padding: 20px 12px; background-color: lightcyan; color: darkcyan; font-weight: bold">
    For each day, there are 90 occurrences.
</div>

# 2 Data Cleaning

In [16]:
print("Missing values in train set: \n", train.isnull().sum()) 
print("\n\nMissing values in test set: \n", test.isnull().sum())

Missing values in train set: 
 id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64


Missing values in test set: 
 id         0
date       0
country    0
store      0
product    0
dtype: int64


So, the missing values are only in the num_sold column

In [25]:
# Group by store-country-product and count missing values
missing_by_group = train.groupby(['store', 'country', 'product'])['num_sold'].agg(
    total_count=lambda x: x.size,
    missing_count=lambda x: x.isna().sum()
).reset_index()

# Sort descending to see worst-affected groups
missing_by_group = missing_by_group.sort_values('missing_count', ascending=False)

# Find all groups with missing values
affected_groups = missing_by_group[missing_by_group['missing_count'] > 0]

print(f"Total groups with missing data: {len(affected_groups)}")
affected_groups.head(len(affected_groups))

Total groups with missing data: 9


,store,country,product,total_count,missing_count
0,Discount Stickers,Canada,Holographic Goose,2557,2557
15,Discount Stickers,Kenya,Holographic Goose,2557,2557
75,Stickers for Less,Kenya,Holographic Goose,2557,1358
60,Stickers for Less,Canada,Holographic Goose,2557,1308
45,Premium Sticker Mart,Kenya,Holographic Goose,2557,646
30,Premium Sticker Mart,Canada,Holographic Goose,2557,380
18,Discount Stickers,Kenya,Kerneler,2557,63
19,Discount Stickers,Kenya,Kerneler Dark Mode,2557,1
3,Discount Stickers,Canada,Kerneler,2557,1


<details>
<summary>Let's break "handling the missing values" down <b>conceptually</b> in three clear steps:</summary>
    <h4><strong>1. Categorize Missing Values by Nature</strong></h4>
    <p>Based on the table, there are <strong>three distinct types</strong> of missing values:</p>

<table>
    <tr>
        <th>Type</th>
        <th>Example Groups</th>
        <th>Missing Pattern</th>
        <th>Likely Cause</th>
    </tr>
    <tr>
        <td><strong>A</strong></td>
        <td>Discount Stickers (Canada/Kenya) - Holographic Goose</td>
        <td>100% missing (2557/2557 days)</td>
        <td>Product <strong>never available</strong> during 2010-2016</td>
    </tr>
    <tr>
        <td><strong>B</strong></td>
        <td>Stickers for Less (Kenya/Canada) - Holographic Goose</td>
        <td>~50% missing</td>
        <td><strong>Temporary unavailability</strong> or data collection gaps</td>
    </tr>
    <tr>
        <td><strong>C</strong></td>
        <td>Kerneler products in Kenya/Canada</td>
        <td>1-63 missing days</td>
        <td><strong>Random gaps</strong> (human error, system glitches)</td>
    </tr>
</table>

<br>

<h4><strong>2. Handle Each Type Differently</strong></h4>
<h5><strong>Type A (Systemic Unavailability)</strong></h5>
<ul>
    <li><strong>Action:</strong> Treat as <strong>true zeros</strong> (<code>num_sold=0</code>)</li>
    <li><strong>Why:</strong></li>
    <ul>
        <li>Test set includes these groups &rarr; product launches in 2017-2019</li>
        <li>Model needs to learn "no sales until availability starts"</li>
    </ul>
    <li><strong>Add Feature:</strong> <code>is_available</code> (0 for these groups in 2010-2016, 1 elsewhere)</li>
</ul>

<h5><strong>Type B (Temporary Gaps)</strong></h5>
<ul>
    <li><strong>Action:</strong></li>
    <ol>
        <li>Forward-fill short gaps (&lt;7 days)</li>
        <li>For longer gaps, use <strong>same-store-country-product median</strong></li>
    </ol>
    <li><strong>Why:</strong></li>
    <ul>
        <li>Preserves temporal consistency</li>
        <li>Avoids overfitting to noise</li>
    </ul>
    <li><strong>Add Feature:</strong> <code>gap_length</code> (days since last observed sale)</li>
</ul>

<h5><strong>Type C (Random Gaps)</strong></h5>
<ul>
    <li><strong>Action:</strong> Use <strong>linear interpolation</strong> (neighboring days' average)</li>
    <li><strong>Why:</strong></li>
    <ul>
        <li>Minimal impact on overall trends</li>
        <li>Simple and conservative</li>
    </ul>
</ul>

<br>

<h4><strong>3. Critical Feature Engineering</strong></h4>
<p>Add these to help models understand missingness context:</p>
<ol>
    <li><code>is_available</code>: Binary flag for systemic unavailability (Type A)</li>
    <li><code>days_since_last_sale</code>: For Type B gaps &rarr; helps model detect recovery patterns</li>
    <li><code>imputation_method</code>: Categorical feature indicating:</li>
    <ul>
        <li>0: No imputation</li>
        <li>1: Systemic zero (Type A)</li>
        <li>2: Median fill (Type B)</li>
        <li>3: Linear interpolation (Type C)</li>
    </ul>
</ol>

<br>

<h4><strong>Why This Works for MAPE</strong></h4>
<ul>
    <li><strong>Type A:</strong> True zeros avoid infinite MAPE penalties if test set has post-2016 sales</li>
    <li><strong>Type B/C:</strong> Median/interpolation provides stable baselines instead of extreme guesses</li>
    <li><strong>Feature Flags:</strong> Allow models to weight predictions based on data quality</li>
</ul>

</details>